# Imports

In [1]:
import pandas as pd
import numpy as np

# Load in Data

In [ ]:
# Load the CSV file
file_path = "F-F_Research_Data_Factors 3.CSV"

# Read the file, skipping header text rows and limiting to monthly data
ff = pd.read_csv(file_path, skiprows=3, nrows=1182)

# Rename the date column for clarity
ff.rename(columns={"Unnamed: 0": "Date"}, inplace=True)

# Convert Date to string to extract Year and Month
ff['Date'] = ff['Date'].astype(str)
ff['Year'] = ff['Date'].str[:4].astype(int)
ff['Month'] = ff['Date'].str[4:].astype(int)

# Pull the columns we want
ff = ff[['Year', 'Month', 'Mkt-RF', 'RF']]

# Show the updated DataFrame
print(ff.head())


   Year  Month  Mkt-RF    RF
0  1926      7    2.96  0.22
1  1926      8    2.64  0.25
2  1926      9    0.36  0.23
3  1926     10   -3.24  0.32
4  1926     11    2.53  0.31


# Part (A)

In [15]:
# Compute market return
ff["Mkt"] = ff["Mkt-RF"] + ff["RF"]

# Compute average monthly return, volatility, and Sharpe ratio
avg_return = ff["Mkt"].mean()
volatility = ff["Mkt"].std()
sharpe_ratio = avg_return / volatility

print(f"Average Monthly Return: {avg_return:.4f}")
print(f"Volatility: {volatility:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

Average Monthly Return: 0.9557
Volatility: 5.3169
Sharpe Ratio: 0.1798
